In [8]:
!cd py && protoc --python_out=. --proto_path=../proto/ message.proto

In [2]:
import requests
import time
host="192.168.50.116"
r = requests.get(f"http://{host}/start_snoop", {})
r.text
time.sleep(1.1)
r = requests.get(f"http://{host}/get_snoop", {})

In [9]:
import message_pb2

def parse_snoop(data):
	position = 0
	length_field = 0
	while position < len(data):
		length_field = data[position]
		position += 1
		if position + length_field > len(data):
			print("bah!")
			return
		encoded = data[position:position+length_field]
		position += length_field
		s = message_pb2.SnoopData()
		s.ParseFromString(encoded)
		yield s


In [10]:
for s in parse_snoop(r.content):
	print(s)

message {
  prop: 268493404
  extended: true
  value: "Q\350\025\314Nh"
}
metadata {
  recv_ms: 9123
  source: MASTER
}

message {
  prop: 433824616
  extended: true
  value: "\005\204"
}
metadata {
  recv_ms: 9129
  source: MASTER
}

message {
  prop: 317389512
  extended: true
  value: "/[*c"
}
metadata {
  recv_ms: 9134
  source: MASTER
}

message {
  prop: 572
  value: "\000"
}
metadata {
  recv_ms: 9140
  source: MASTER
}

message {
  prop: 414240056
  extended: true
  value: "r"
}
metadata {
  recv_ms: 9145
  source: MASTER
}

message {
  prop: 5191832
  extended: true
  value: "<"
}
metadata {
  recv_ms: 9150
  source: MASTER
}

message {
  prop: 1696
  value: "\246\342\206\036\275"
}
metadata {
  recv_ms: 9165
  source: MASTER
}

message {
  prop: 1812
  value: "?(\367R>\344\374"
}
metadata {
  recv_ms: 9170
  source: MASTER
}

message {
  prop: 386707912
  extended: true
  value: "~T\026\017\356"
}
metadata {
  recv_ms: 9175
  source: MASTER
}

message {
  prop: 154984144
  ex